In [148]:
import json
import requests
from pprint import pprint
import difflib
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime

In [4]:
with open('data.json') as json_data:
    d = json.load(json_data)
    data = d

In [5]:
len(data)

15

In [6]:
documents = {'documents' : []}
other_docs = {'documents': []}

for store in data:
    text = ''
    for i in store['reviews']:
        text += i[0]
        
    if(len(text) > 5120):
        done = False
        start = 0
        iteration = 0
        while(start < len(text)):
            end = start + 5120
            if(end >= len(text)):
                end = len(text)
            print(start, end)
            documents['documents'].append({'id': store['id'] +';' + str(iteration), 'language': 'en',
                                   'text': text[start: end]})
            iteration+=1
            start = end
    else:
        documents['documents'].append({'id': store['id'], 'language': 'en', 'text': text})
    
    other_docs['documents'].append({'id': store['id'], 'language': 'en', 'text': text})
    
    print('finished store')

0 5120
5120 10240
10240 13980
finished store
0 5120
5120 10240
10240 12342
finished store
0 5120
5120 10240
10240 10635
finished store
0 5120
5120 5270
finished store
finished store
finished store
0 5120
5120 7745
finished store
finished store
finished store
finished store
finished store
finished store
finished store
finished store
finished store


In [9]:
def getPopularPhones(business_id):
    pop_phones = []
    
    for store in key_phrases['documents']:
        if(business_id in store['id']):
            phones = {
                'samsung': [
                    's9',
                    's8',
                    's7',
                    'note',
                    's6', 
                ],
                'apple': ['iphone', 'ipad', 'ipod'],
                'motorola': ['motoX'],
                'google': ['Nexus'],
                'LG': ['g9', 'g5', 'g6', 'g3', 'g4'],
            }

            phrases = store['keyPhrases']
            frequencies = {}
            for word in phrases:
                for brand,v in phones.items():
                    for phone in v:
                        seq = difflib.SequenceMatcher(None,word.lower(),phone)
                        d = seq.ratio()*100
                        if(d > 0.75):
                            largest = None
                            smallest = None
                            if(len(phone) > len(word)):
                                largest = phone
                                smallest = word
                            else:
                                largest = word
                                smallest = phone

                            if(smallest.lower() in largest.lower()):
                                if(brand in frequencies.keys()):
                                    frequencies[phone] += 1
                                else:
                                    frequencies[phone] = 1
    return frequencies


text_analytics_base_url = "https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0"
key_phrase_api_url = text_analytics_base_url + "/keyPhrases"

headers   = {'Ocp-Apim-Subscription-Key': '5c0a1002f18a454c9ebadfe86b72964f'}
response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
key_phrases = response.json()

#print(key_phrases['documents'])
#print(getPopularPhones('N_v0tMRmqgTnnusrcVI3dw'))
for i in key_phrases['documents']:
    print(getPopularPhones(i['id']), i['id'])

{} wKKRoSg1pZQtQRYPoD7TXg;0
{} wKKRoSg1pZQtQRYPoD7TXg;1
{'iphone': 1} wKKRoSg1pZQtQRYPoD7TXg;2
{} 8hXzJzOb3K1eFor0oNrPWA;0
{} 8hXzJzOb3K1eFor0oNrPWA;1
{'iphone': 1, 'note': 1} 8hXzJzOb3K1eFor0oNrPWA;2
{'s7': 1, 'iphone': 1, 'note': 1} z-ldpf_9VxZjITfBb7Lnkg;0
{'Nexus': 1} z-ldpf_9VxZjITfBb7Lnkg;1
{} z-ldpf_9VxZjITfBb7Lnkg;2
{'s6': 1, 'iphone': 1} qiAckHMVZQScy2U9nvSCmQ;0
{} qiAckHMVZQScy2U9nvSCmQ;1
{'iphone': 1} 9oHGS1knSS3uAHsf2sqxRg
{'iphone': 1} oKrbc2dBvop1eo4dwdGk6Q
{'iphone': 1} GP2c4RUrfCPivCTVZ_VueA;0
{} GP2c4RUrfCPivCTVZ_VueA;1
{'s6': 1, 'iphone': 1} ZgkdwRzfT2QjQbzi34t96A
{'iphone': 1} N_v0tMRmqgTnnusrcVI3dw
{'iphone': 1, 'Nexus': 1} EGHoMbnG-vzwotSQz9oeqQ
{} 8XBfdAwp1Y_IrhFdSZaNzw
{'iphone': 1} Qh0Z1gyxugwZzAna4UsCKA
{} sSc4MPlUOPCEXEDQw_DISg
{'motoX': 1} jMCpYt9N6VqtjS-U2ubjZQ
{'note': 1} RlaaCtbydgyg8ItxIeuFRQ


In [165]:
import json
import os
import nltk
from nltk import FreqDist
import pandas as pd
import matplotlib.pyplot as plt
import re
import requests
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import time

subscription_key = '5c0a1002f18a454c9ebadfe86b72964f'  # os.environ['MS_KEY']
assert subscription_key
text_analytics_base_url = "https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0/"

class StoreDB:
    def __init__(self):
        self.df = pd.read_json('../datascraper/data.json', dtype=object)
        self.df = self.df.set_index('name')
        self.df['agg_sent'] = 0.0
        self.df['sentiments'] = None
        self.df['time'] = None
        self.df['sentiments'] = self.df['sentiments'].astype(object)
        self.df['key_phrases_pos'] = None
        self.df['key_phrases_pos'] = self.df['key_phrases_pos'].astype(object)
        self.df['key_phrases_neg'] = None
        self.df['key_phrases_neg'] = self.df['key_phrases_neg'].astype(object)
        for index, row in self.df.iterrows():
            idnum = 0
            docDic = []
            times = []
            for rev in self.df.loc[index]['reviews']:
                docDic.append({'id':idnum, 'language':'en', 'text': rev[0]})
                try:
                    times.append([rev[1], datetime.strptime(rev[2].rstrip().lstrip(), "%m/%d/%Y")])
                except:
                    pass
                idnum += 1
            
            self.df.at[index, 'time'] = times
            
            documents = {'documents' : docDic}
            headers   = {'Ocp-Apim-Subscription-Key': subscription_key}
            sentiment_api_url = text_analytics_base_url + "sentiment"
            response  = requests.post(sentiment_api_url, headers=headers, json=documents)
            sentiments = response.json()
            sent_scores = []
            pos_docs = []
            neg_docs = []
            try:
                for doc in sentiments['documents']:
                    val = doc['score']
                    if val > 0.5:
                        pos_docs.append(doc['id'])
                    else:
                        neg_docs.append(doc['id'])
                    sent_scores.append(doc['score'])
            except:
                print(documents)
                print(sentiments)
            self.df.at[index, 'agg_sent'] = sum(sent_scores) / float(len(sent_scores))
            self.df.at[index, 'sentiments'] = sentiments['documents']

            key_phrase_api_url = text_analytics_base_url + "keyPhrases"
            headers   = {'Ocp-Apim-Subscription-Key': subscription_key}
            response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
            key_phrases = response.json()
            pos_phrases = []
            neg_phrases = []
            for doc in key_phrases['documents']:
                if doc['id'] in pos_docs:
                    pos_phrases.append(doc)
                elif doc['id'] in neg_docs:
                    neg_phrases.append(doc)
                    

            self.df.at[index, 'key_phrases_pos'] = pos_phrases
            self.df.at[index, 'key_phrases_neg'] = neg_phrases

    def key_phrases_pos(self, store):
        pos_phrases = self.df.loc[store]['key_phrases_pos']
        return pos_phrases
    
    def key_phrases_neg(self, store):
        neg_phrases = self.df.loc[store]['key_phrases_neg']
        return neg_phrases 

    def get_raw_reviews(self, store):
        raw_revs = self.df.loc[store]['reviews']
        return raw_revs
    
    def get_agg_sent(self, store):
        return self.df.loc[store]['aggSent']    

In [166]:
db = StoreDB()

In [189]:
db.df

,id,location,reviews,agg_sent,sentiments,time,key_phrases_pos,key_phrases_neg
name,,,,,,,,
t-mobile-atlanta-13,wKKRoSg1pZQtQRYPoD7TXg,"{'zip_code': '30309', 'state': 'GA', 'country'...",[[Was in town for work and had issues with my ...,0.322871,"[{'score': 0.9026597738265991, 'id': '0'}, {'s...","[[5.0, 2019-01-29 00:00:00], [5.0, 2018-11-08 ...","[{'id': '0', 'keyPhrases': ['issues', 'right',...","[{'id': '1', 'keyPhrases': ['upgrade process',..."
t-mobile-atlanta-3,8hXzJzOb3K1eFor0oNrPWA,"{'zip_code': '30305', 'state': 'GA', 'country'...",[[The customer service is off the charts!!! Th...,0.407450,"[{'score': 0.8873186111450195, 'id': '0'}, {'s...","[[5.0, 2019-03-02 00:00:00], [1.0, 2018-12-10 ...","[{'id': '0', 'keyPhrases': ['customer service'...","[{'id': '1', 'keyPhrases': ['Complaint', 'time..."
t-mobile-atlanta-39,z-ldpf_9VxZjITfBb7Lnkg,"{'zip_code': '30346', 'state': 'GA', 'country'...",[[If you haven't been to see us in over a year...,0.266538,"[{'score': 0.9040725827217102, 'id': '0'}, {'s...","[[5.0, 2018-11-24 00:00:00], [1.0, 2018-06-14 ...","[{'id': '0', 'keyPhrases': ['remodeled store',...","[{'id': '1', 'keyPhrases': ['bad review', 'cha..."
t-mobile-austell,qiAckHMVZQScy2U9nvSCmQ,"{'zip_code': '30106', 'state': 'GA', 'country'...",[[Kudos to Tatiana! My daughter need a screen ...,0.485955,"[{'score': 0.9516618251800537, 'id': '0'}, {'s...","[[5.0, 2018-10-18 00:00:00], [4.0, 2016-10-18 ...","[{'id': '0', 'keyPhrases': ['screen cover', 'd...","[{'id': '3', 'keyPhrases': ['new sim card', 'T..."
t-mobile-lilburn-2,9oHGS1knSS3uAHsf2sqxRg,"{'zip_code': '30047', 'state': 'GA', 'country'...",[[I would like to start by saying I was pleasa...,0.224441,"[{'score': 0.8953238725662231, 'id': '0'}, {'s...","[[5.0, 2018-03-26 00:00:00], [1.0, 2018-12-28 ...","[{'id': '0', 'keyPhrases': ['rep', 'location',...","[{'id': '1', 'keyPhrases': ['T-Mobile cop', 'p..."
t-mobile-atlanta-30,oKrbc2dBvop1eo4dwdGk6Q,"{'zip_code': '30308', 'state': 'GA', 'country'...",[[I had a great experience switching from AT&T...,0.609104,"[{'score': 0.8545178174972534, 'id': '0'}, {'s...","[[4.0, 2019-02-07 00:00:00], [1.0, 2017-12-13 ...","[{'id': '0', 'keyPhrases': ['old phone', 'new ...","[{'id': '1', 'keyPhrases': ['store', 'Staff', ..."
t-mobile-atlanta-23,GP2c4RUrfCPivCTVZ_VueA,"{'zip_code': '30346', 'state': 'GA', 'country'...",[[Let's start this one off with a good ole' GR...,0.327776,"[{'score': 0.05033788084983826, 'id': '0'}, {'...","[[2.0, 2013-03-18 00:00:00], [1.0, 2019-02-25 ...","[{'id': '3', 'keyPhrases': ['new T-Mobile cust...","[{'id': '0', 'keyPhrases': ['temporary phone',..."
t-mobile-atlanta-46,ZgkdwRzfT2QjQbzi34t96A,"{'zip_code': '30324', 'state': 'GA', 'country'...","[[You're like, ""Why the hell is he reviewing a...",0.466963,"[{'score': 0.8085744380950928, 'id': '0'}, {'s...","[[4.0, 2017-03-31 00:00:00], [1.0, 2017-09-29 ...","[{'id': '0', 'keyPhrases': ['T-Mobile store', ...","[{'id': '1', 'keyPhrases': ['total indifferenc..."
t-mobile-lithia-springs,N_v0tMRmqgTnnusrcVI3dw,"{'zip_code': '30122', 'state': 'GA', 'country'...",[[We came to this T-Mobile location about a mo...,0.329740,"[{'score': 0.07720571756362915, 'id': '0'}, {'...","[[3.0, 2019-03-13 00:00:00], [1.0, 2019-03-28 ...","[{'id': '1', 'keyPhrases': ['Poor customer ser...","[{'id': '0', 'keyPhrases': ['phone number', 'T..."


In [72]:
from math import pi

import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.palettes import Category20c, viridis
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.palettes import Spectral6
from bokeh.models import ColumnDataSource


In [118]:
class GraphicsLib:
    def __init__(self, business):
        text_analytics_base_url = "https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0"
        key_phrase_api_url = text_analytics_base_url + "/keyPhrases"

        headers   = {'Ocp-Apim-Subscription-Key': '5c0a1002f18a454c9ebadfe86b72964f'}
        response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
        self.key_phrases = response.json()
        self.id = business
        self.sid = SentimentIntensityAnalyzer()
        
    def getPopularPhones(self):
        pop_phones = []
        
        business_id = self.id
        for store in self.key_phrases['documents']:
            if(business_id in store['id']):
                phones = {
                    'samsung': [
                        's9',
                        's8',
                        's7',
                        'note',
                        's6', 
                    ],
                    'apple': ['iphone', 'ipad', 'ipod'],
                    'motorola': ['motoX'],
                    'google': ['Nexus'],
                    'LG': ['g9', 'g5', 'g6', 'g3', 'g4'],
                }

                phrases = store['keyPhrases']
                frequencies = {}
                for word in phrases:
                    for brand,v in phones.items():
                        for phone in v:
                            seq = difflib.SequenceMatcher(None,word.lower(),phone)
                            d = seq.ratio()*100
                            if(d > 0.75):
                                largest = None
                                smallest = None
                                if(len(phone) > len(word)):
                                    largest = phone
                                    smallest = word
                                else:
                                    largest = word
                                    smallest = phone

                                if(smallest.lower() in largest.lower()):
                                    if(brand in frequencies.keys()):
                                        frequencies[phone] += 1
                                    else:
                                        frequencies[phone] = 1
        return frequencies
    
    def getServiceSentiment(self):
        business_id = self.id
        sentiment = {'pos': [0, []], 'neg': [0, []], 'neutral': [0, []]}
        for store in self.key_phrases['documents']:
            if(business_id in store['id']):
                phrases = store['keyPhrases']
                frequencies = {}
                phones = ['customer', 'service']
                for phone in phones:
                    for word in phrases:
                        seq = difflib.SequenceMatcher(None,word.lower(),phone)
                        d = seq.ratio()*100
                        if(d > 0.75):
                            largest = None
                            smallest = None
                            if(len(phone) > len(word)):
                                largest = phone
                                smallest = word
                            else:
                                largest = word
                                smallest = phone

                            if(smallest.lower() in largest.lower()):
                                polarity = self.sid.polarity_scores(largest)['compound']
                                if(polarity == 0.0):
                                    sentiment['neutral'][0] += 1
                                    sentiment['neutral'][1].append(largest)
                                elif(polarity < 0.0):
                                    sentiment['neg'][0] += 1
                                    sentiment['neg'][1].append(largest)
                                else:
                                    sentiment['pos'][0] += 1
                                    sentiment['pos'][1].append(largest)
                                    
        return sentiment
                    
    def getPhoneChart(self):
        x = self.getPopularPhones()
        
        data = pd.Series(x).reset_index(name='value').rename(columns={'index':'models'})
        data['angle'] = data['value']/data['value'].sum() * 2*pi
        data['color'] = viridis(len(x))

        p = figure(plot_height=350, title="Phone useage in Store %s" % (store_id), toolbar_location=None,
                   tools="hover", tooltips="@models: @value", x_range=(-0.5, 1.0))

        p.wedge(x=0, y=1, radius=0.4,
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                line_color="white", fill_color='color', legend='models', source=data)

        p.axis.axis_label=None
        p.axis.visible=False
        p.grid.grid_line_color = None

        output_notebook()
        show(p)
        
    def getServiceSentimentChart(self):
        x = self.getServiceSentiment()

        source = ColumnDataSource(data=dict(sentiment= list(x.keys()), counts= [v[0] for k,v in x.items()],
                                            color=Spectral6, services = [';'.join(v[1]) for k,v in x.items()]))

        p = figure(x_range = list(x.keys()), title="Service Sentiment for %s" % (self.id),
                   toolbar_location=None, tools="hover", tooltips="@sentiment: @services")

        p.vbar(x='sentiment', top='counts', width=0.9, color='color', source=source)

        p.xgrid.grid_line_color = None
        
        output_notebook()
        show(p)
        
    def returnData(self):
        return self.key_phrases
    
    def getStoreIds(self):
        return list(map(lambda x: x['id'], self.returnData()['documents']))
                

In [178]:
class ChefJeff:
    def __init__(self, db, business):
        self.db = db
        self.id = business
        self.sid = SentimentIntensityAnalyzer()
        
    def getPopularPhones(self):
        pop_phones = []
        phones = {
                    'samsung': [
                        's9',
                        's8',
                        's7',
                        'note',
                        's6', 
                    ],
                    'apple': ['iphone', 'ipad', 'ipod'],
                    'motorola': ['motoX'],
                    'google': ['Nexus'],
                    'LG': ['g9', 'g5', 'g6', 'g3', 'g4'],
                }
        frequencies = {}
        
        business_id = self.id
        data = self.db.df.loc[business_id]
        for category in ['key_phrases_pos', 'key_phrases_neg']:
            for dictionary in data[category]:
                for word in dictionary['keyPhrases']:
                    for brand,v in phones.items():
                        for phone in v:
                            seq = difflib.SequenceMatcher(None,word.lower(),phone)
                            d = seq.ratio()*100
                            if(d > 0.75):
                                largest = None
                                smallest = None
                                if(len(phone) > len(word)):
                                    largest = phone
                                    smallest = word
                                else:
                                    largest = word
                                    smallest = phone

                                if(smallest.lower() in largest.lower()):
                                    if(brand in frequencies.keys()):
                                        frequencies[phone] += 1
                                    else:
                                        frequencies[phone] = 1
        return frequencies
    
    def getServiceSentiment(self):
        sentiment = {'pos': [0, []], 'neg': [0, []], 'neutral': [0, []]}
        
        business_id = self.id
        data = self.db.df.loc[business_id]
        for category in ['key_phrases_pos', 'key_phrases_neg']:
            for dictionary in data[category]:
                phones = ['customer', 'service']
                for phone in phones:
                    for word in dictionary['keyPhrases']:
                        seq = difflib.SequenceMatcher(None,word.lower(),phone)
                        d = seq.ratio()*100
                        if(d > 0.75):
                            largest = None
                            smallest = None
                            if(len(phone) > len(word)):
                                largest = phone
                                smallest = word
                            else:
                                largest = word
                                smallest = phone

                            if(smallest.lower() in largest.lower()):
                                polarity = self.sid.polarity_scores(largest)['compound']
                                if(polarity == 0.0):
                                    sentiment['neutral'][0] += 1
                                    sentiment['neutral'][1].append(largest)
                                elif(polarity < 0.0):
                                    sentiment['neg'][0] += 1
                                    sentiment['neg'][1].append(largest)
                                else:
                                    sentiment['pos'][0] += 1
                                    sentiment['pos'][1].append(largest)
                                    
        return sentiment
                    
    def getPhoneChart(self):
        x = self.getPopularPhones()
        
        data = pd.Series(x).reset_index(name='value').rename(columns={'index':'models'})
        data['angle'] = data['value']/data['value'].sum() * 2*pi
        data['color'] = viridis(len(x))

        p = figure(plot_height=350, title="Phone useage in Store %s" % (store_id), toolbar_location=None,
                   tools="hover", tooltips="@models: @value", x_range=(-0.5, 1.0))

        p.wedge(x=0, y=1, radius=0.4,
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                line_color="white", fill_color='color', legend='models', source=data)

        p.axis.axis_label=None
        p.axis.visible=False
        p.grid.grid_line_color = None

        output_notebook()
        show(p)
        
    def getServiceSentimentChart(self):
        x = self.getServiceSentiment()

        source = ColumnDataSource(data=dict(sentiment= list(x.keys()), counts= [v[0] for k,v in x.items()],
                                            color=Spectral6, services = [';'.join(v[1]) for k,v in x.items()]))

        p = figure(x_range = list(x.keys()), title="Service Sentiment for %s" % (self.id),
                   toolbar_location=None, tools="hover", tooltips="@sentiment: @services")

        p.vbar(x='sentiment', top='counts', width=0.9, color='color', source=source)

        p.xgrid.grid_line_color = None
        
        output_notebook()
        show(p)
    
    def getReviewsOverTime(self):
        
        data = self.db.df.loc[self.id]['time']
        
        source = ColumnDataSource(data=dict(time = [i[1] for i in data], rating = [i[0] for i in data]))

        p = figure(x_axis_type='datetime',
                   x_axis_label = 'time',
                   y_axis_label = 'ratings',
                   title = "Ratings of %s Over Time" % (self.id),
                   plot_width = 1100
                   )

        p.circle(x='time', y='rating', color='blue', source=source)

        p.xgrid.grid_line_color = None
        
        output_notebook()
        show(p)
        
    def returnData(self):
        return self.key_phrases
    
    def getStoreIds(self):
        return list(map(lambda x: x['id'], self.returnData()['documents']))
                

In [179]:
a = ChefJeff(db, 't-mobile-atlanta-3')
a.getReviewsOverTime()

Loading BokehJS ...

In [119]:
a = GraphicsLib('z-ldpf_9VxZjITfBb7Lnkg')
a.getServiceSentimentChart()
print(a.getStoreIds())

Loading BokehJS ...

['wKKRoSg1pZQtQRYPoD7TXg;0', 'wKKRoSg1pZQtQRYPoD7TXg;1', 'wKKRoSg1pZQtQRYPoD7TXg;2', '8hXzJzOb3K1eFor0oNrPWA;0', '8hXzJzOb3K1eFor0oNrPWA;1', '8hXzJzOb3K1eFor0oNrPWA;2', 'z-ldpf_9VxZjITfBb7Lnkg;0', 'z-ldpf_9VxZjITfBb7Lnkg;1', 'z-ldpf_9VxZjITfBb7Lnkg;2', 'qiAckHMVZQScy2U9nvSCmQ;0', 'qiAckHMVZQScy2U9nvSCmQ;1', '9oHGS1knSS3uAHsf2sqxRg', 'oKrbc2dBvop1eo4dwdGk6Q', 'GP2c4RUrfCPivCTVZ_VueA;0', 'GP2c4RUrfCPivCTVZ_VueA;1', 'ZgkdwRzfT2QjQbzi34t96A', 'N_v0tMRmqgTnnusrcVI3dw', 'EGHoMbnG-vzwotSQz9oeqQ', '8XBfdAwp1Y_IrhFdSZaNzw', 'Qh0Z1gyxugwZzAna4UsCKA', 'sSc4MPlUOPCEXEDQw_DISg', 'jMCpYt9N6VqtjS-U2ubjZQ', 'RlaaCtbydgyg8ItxIeuFRQ']


In [109]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Sachin
[nltk_data]     Konan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [110]:
from nltk.tokenize import sent_tokenize, word_tokenize 
  
def tokenize(text):
    return word_tokenize(text)

In [111]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to C:\Users\Sachin
[nltk_data]     Konan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [112]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Sachin
[nltk_data]     Konan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [113]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [114]:
import random
text_data = []

for i in range(0, len(other_docs['documents'])):
    string = other_docs['documents'][i]['text']
    tokens = prepare_text_for_lda(string)
    if random.random() > .75:
        print(tokens)
        text_data.append(tokens)
        
print(text_data)

['issue', 'phone', 'service', 'T-mobile', 'earliest', 'right', 'midtown', 'right', 'customer', 'service', 'knowledgeable', 'friendly', 'help', 'fix', 'issue', 'recommend', 'great', 'restaurant', 'meeting', 'topic', 'Thank', '5Church', 'great', 'suggestion', 'Thanks', 'upgrade', 'month', 'There', 'really', 'anyone', 'quiet', 'place', 'organize', 'associate', 'helpful', 'upgrade', 'process', 'Although', 'trouble', 'try', 'device', 'TeamMagenta', 'answer', 'question', 'going', 'upgrade', 'walk', 'upgrade', 'phone', 'ask', 'would', 'transfer', 'back', '10-15', 'minutes', 'helping', 'texting', 'helping', 'screw', 'transfer', 'times', 'ask', 'manager', 'either', 'store', 'hours', 'Terrible', 'service', 'extremely', 'unsatisfied', 'would', 'recommend', 'coming', 'location', 'better', 'service', 'hopefully', 'better', 'T-mobile', 'location', 'actually', 'know', 'doing.Maybe', 'lucky', 'young', 'name', "'Houston", 'super', 'friendly', 'hook', 'flash', 'friendly', 'accommodate', 'need', 'extreme

In [115]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [117]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=NUM_TOPICS)
for topic in topics:
    print(topic)

(0, '0.001*"T-Mobile" + 0.001*"service" + 0.001*"phone" + 0.001*"location" + 0.001*"customer" + 0.001*"experience" + 0.001*"store" + 0.001*"would" + 0.001*"month" + 0.001*"could"')
(1, '0.028*"phone" + 0.024*"service" + 0.022*"customer" + 0.022*"T-Mobile" + 0.022*"store" + 0.015*"location" + 0.014*"would" + 0.011*"could" + 0.011*"experience" + 0.008*"month"')
(2, '0.001*"store" + 0.001*"customer" + 0.001*"phone" + 0.001*"service" + 0.001*"would" + 0.001*"T-Mobile" + 0.001*"could" + 0.001*"manager" + 0.001*"explain" + 0.001*"employee"')
(3, '0.001*"service" + 0.001*"phone" + 0.001*"customer" + 0.001*"store" + 0.001*"T-Mobile" + 0.001*"would" + 0.001*"could" + 0.001*"manager" + 0.001*"experience" + 0.001*"employee"')
(4, '0.001*"store" + 0.001*"service" + 0.001*"customer" + 0.001*"T-Mobile" + 0.001*"phone" + 0.001*"location" + 0.001*"would" + 0.001*"employee" + 0.001*"could" + 0.001*"manager"')
(5, '0.029*"phone" + 0.029*"store" + 0.024*"customer" + 0.016*"service" + 0.012*"would" + 0.01

In [183]:
import pandas as pd

import requests

from datetime import datetime
import os

subscription_key = 'c1005ffa18cc42ddbf916d2483bb2dd2'  # os.environ['MS_KEY']
assert subscription_key
text_analytics_base_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.0/"

class StoreDB:
    def __init__(self):
        if(os.path.isfile('../datascraper/reviews.pkl')):
            self.df = pd.read_pickle("../datascraper/reviews.pkl")
        else:
            self.df = pd.read_json('../datascraper/data.json', dtype=object)
            self.df = self.df.set_index('name')
            self.df['agg_sent'] = 0.0
            self.df['sentiments'] = None
            self.df['time'] = None
            self.df['sentiments'] = self.df['sentiments'].astype(object)
            self.df['key_phrases_pos'] = None
            self.df['key_phrases_pos'] = self.df['key_phrases_pos'].astype(object)
            self.df['key_phrases_neg'] = None
            self.df['key_phrases_neg'] = self.df['key_phrases_neg'].astype(object)
            for index, row in self.df.iterrows():
                idnum = 0
                docDic = []
                times = []
                for rev in self.df.loc[index]['reviews']:
                    docDic.append({'id':idnum, 'language':'en', 'text': rev[0]})
                    try:
                        times.append([rev[1], datetime.strptime(rev[2].rstrip().lstrip(), "%m/%d/%Y")])
                    except:
                        pass
                    idnum += 1

                self.df.at[index, 'time'] = times

                documents = {'documents' : docDic}
                headers   = {'Ocp-Apim-Subscription-Key': subscription_key}
                sentiment_api_url = text_analytics_base_url + "sentiment"
                response  = requests.post(sentiment_api_url, headers=headers, json=documents)
                sentiments = response.json()
                sent_scores = []
                pos_docs = []
                neg_docs = []
                try:
                    for doc in sentiments['documents']:
                        val = doc['score']
                        if val > 0.5:
                            pos_docs.append(doc['id'])
                        else:
                            neg_docs.append(doc['id'])
                        sent_scores.append(doc['score'])
                except:
                    print(documents)
                    print(sentiments)
                self.df.at[index, 'agg_sent'] = sum(sent_scores) / float(len(sent_scores))
                self.df.at[index, 'sentiments'] = sentiments['documents']

                key_phrase_api_url = text_analytics_base_url + "keyPhrases"
                headers   = {'Ocp-Apim-Subscription-Key': subscription_key}
                response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
                key_phrases = response.json()
                pos_phrases = []
                neg_phrases = []
                for doc in key_phrases['documents']:
                    if doc['id'] in pos_docs:
                        pos_phrases.append(doc)
                    elif doc['id'] in neg_docs:
                        neg_phrases.append(doc)


                self.df.at[index, 'key_phrases_pos'] = pos_phrases
                self.df.at[index, 'key_phrases_neg'] = neg_phrases
    def key_phrases_pos(self, store):
        pos_phrases = self.df.loc[store]['key_phrases_pos']
        return pos_phrases
    
    def key_phrases_neg(self, store):
        neg_phrases = self.df.loc[store]['key_phrases_neg']
        return neg_phrases 

    def get_raw_reviews(self, store):
        raw_revs = self.df.loc[store]['reviews']
        return raw_revs

    def get_sentiments(self, store):
        return self.df.loc[store]['sentiments']

    def get_indexed_sent(self, store, id):
        sents = self.get_sentiments(store)
        for doc in sents:
            if doc['id'] == id:
                return doc['score']

    def get_avg_stars(self, store):
        return self.df.loc[store]['stars']

    def get_indexed_review(self, store, id):
        raw_rev = self.df.loc[store]['reviews'][id][0]
        return raw_rev

    def get_indexed_pos_keyPhrases(self, store, id):
        pos_phrase_all = self.key_phrases_pos(store)
        for doc in pos_phrase_all:
            if doc['id'] == id:
                return doc['keyPhrases']

    def get_indexed_neg_keyPhrases(self, store, id):
        neg_phrase_all = self.key_phrases_neg(store)
        for doc in neg_phrase_all:
            if doc['id'] == id:
                return doc['keyPhrases']

    def get_agg_sent(self, store):
        return self.df.loc[store]['aggSent']

    def keyword_search(self, store, word):
        sentiments4word = []
        pos_phrases = self.key_phrases_pos(store)
        neg_phrases = self.key_phrases_neg(store)

        for doc in pos_phrases:
            if word in doc['keyPhrases']:
                sentiments4word.append(self.get_indexed_sent(store, doc['id']))
        
        return sentiments4word